# Selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
 
options = Options()
options.add_argument("--disable-notifications")
 
chrome = webdriver.Chrome('./chromedriver', options=options)
chrome.get("https://www.facebook.com/")
 
email = chrome.find_element_by_id("email")
password = chrome.find_element_by_id("pass")
 
email.send_keys('ACCOUNT')
password.send_keys('PASSWORD')
password.submit()

ModuleNotFoundError: No module named 'selenium'

In [2]:
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import time
import os
import sys


def Get_StockPrice(Symbol, Name, Date):

    url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={Date}&stockNo={Symbol}'
    print(url)
    data = requests.get(url).text
    json_data = json.loads(data)

    Stock_data = json_data['data']

    StockPrice = pd.DataFrame(Stock_data, columns=[
                              'Date', 'Volume', 'Volume_Cash', 'Open', 'High', 'Low', 'Close', 'Change', 'Order'])

    StockPrice['Date'] = StockPrice['Date'].str.replace(
        '/', '').astype(int) + 19110000
    StockPrice['Date'] = pd.to_datetime(StockPrice['Date'].astype(str))
    StockPrice['Volume'] = StockPrice['Volume'].str.replace(
        ',', '').astype(float)/1000
    StockPrice['Volume_Cash'] = StockPrice['Volume_Cash'].str.replace(
        ',', '').astype(float)
    StockPrice['Order'] = StockPrice['Order'].str.replace(
        ',', '').astype(float)

    StockPrice['Open'] = StockPrice['Open'].str.replace(',', '').astype(float)
    StockPrice['High'] = StockPrice['High'].str.replace(',', '').astype(float)
    StockPrice['Low'] = StockPrice['Low'].str.replace(',', '').astype(float)
    StockPrice['Close'] = StockPrice['Close'].str.replace(
        ',', '').astype(float)

    StockPrice['ID'] = Symbol
    StockPrice['Name'] = Name

    StockPrice = StockPrice.set_index('Date', drop=True)

    StockPrice = StockPrice[['ID', "Name", 'Open',
                             'High', 'Low', 'Close', 'Volume']]
#     print(StockPrice)
    return StockPrice


if __name__ == '__main__':
    start_date = '2021-01-01'
    today = str(date.today())

    if os.path.isfile("stock.csv"):
        stock_record = pd.read_csv("stock.csv")
        last = stock_record["Date"][-1:].to_string(index=False).strip()
        last_date = datetime.strptime(last, '%Y-%m-%d').strftime('%Y-%m-%d')

        if str(last_date) == today:
            print("今日已執行過爬蟲")
            sys.exit(0)
        elif last_date > datetime.strptime(start_date, '%Y-%m-%d').strftime('%Y-%m-%d'):
            start_date = datetime.strptime(last, '%Y-%m-%d').replace(day=1)
    else:
        last_date = datetime.strptime(start_date, '%Y-%m-%d').replace(day=2)
        last_date = (last_date - relativedelta(months=1)).strftime('%Y-%m-%d')
        if start_date == date.today():
            start_date = (datetime.now()+timedelta(days=-1)).strftime("%Y-%m-%d")

    # 讀取上市公司的列表檔案，取出公司名稱以及公司代號
    stock_df = pd.read_csv("StockList.csv")
    stock_list = list(stock_df["代號"].apply(
        lambda x: x.replace('"', "").replace('=', "")))
    name = list(stock_df["名稱"])

    stock_list = list(zip(stock_list, name))[0:10]  # 取出前10家公司測試
    Dates = pd.date_range(start=start_date,
                          end=today, freq='MS').astype(str)

    # 迭代取出 stock_list 中的公司名稱以及代號，並呼叫 Get_StockPrice
    for Symbol, Name in stock_list:
        print(Dates[0])
        data = Get_StockPrice(Symbol, Name, Dates[0].replace('-', ''))
        time.sleep(2)
        
        for Date in Dates[1:]:
            
            try:
                print(Date)
                data = pd.concat([data, Get_StockPrice(
                    Symbol, Name, Date.replace('-', ''))], axis=0)
                time.sleep(2)
            except:
                pass

    # if file does not exist write header
        if not os.path.isfile("stock.csv"):
            data = data[data.index > last_date]
            data.to_csv("stock.csv")
        else:  # else it exists so append without writing the header
            data = data[data.index > last_date]
            data.to_csv("stock.csv", mode='a', header=False)
        if len(data) != 0:
            print(data)
        else:
            print(f"All records of {Name} have been updated!")

FileNotFoundError: [Errno 2] No such file or directory: 'StockList.csv'